In [1]:
import json
import os
import requests
import random
import string
import secrets
import time
import re
import collections
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
import copy
import numpy as np
from collections import defaultdict
from collections import Counter

In [2]:

class HangmanDataset(Dataset):
    def __init__(self, words, max_word_length=45, reveal_ratio=0.5):
        self.words = [word.lower() for word in words if len(word) <= max_word_length]
        self.max_length = max_word_length
        self.reveal_ratio = reveal_ratio
        self.char_to_idx = {char: i+1 for i, char in enumerate(string.ascii_lowercase)}
        self.char_to_idx['_'] = 0  # blank
        self.char_to_idx['PAD'] = 27

    def __len__(self): return len(self.words) * 80

    def __getitem__(self, idx):
        word = self.words[idx % len(self.words)]
        reveal_count = int(len(word) * self.reveal_ratio)
        revealed = random.sample(range(len(word)), reveal_count) if reveal_count > 0 else []

        word_state = [0] * self.max_length
        for pos in revealed: word_state[pos] = self.char_to_idx[word[pos]]

        target_pos, target_chars, position_context, vowels = [], [], [0]*self.max_length, set('aeiou')
        for i in range(len(word)):
            if i not in revealed:
                ctx = 0
                if i > 0 and word_state[i-1] != 0: ctx += 1
                if i < len(word)-1 and word_state[i+1] != 0: ctx += 2
                if ctx:
                    target_pos.append(i)
                    target_chars.append(self.char_to_idx[word[i]])
                    position_context[i] = ctx

        count_blanks = word_state[:len(word)].count(0)
        blank_vowel_next = [0]*self.max_length
        for i in range(len(word)):
            if word_state[i] == 0:
                l = word[i-1] if i > 0 else 'x'
                r = word[i+1] if i < len(word)-1 else 'x'
                if l in vowels or r in vowels:
                    blank_vowel_next[i] = 1

        max_targets = 10
        while len(target_pos) < max_targets:
            target_pos.append(-1)
            target_chars.append(0)

        return {
            'word_state': torch.tensor(word_state, dtype=torch.long),
            'position_context': torch.tensor(position_context, dtype=torch.long),
            'target_positions': torch.tensor(target_pos[:max_targets], dtype=torch.long),
            'target_chars': torch.tensor(target_chars[:max_targets], dtype=torch.long),
            'word_length': torch.tensor(len(word), dtype=torch.long),
            'blank_count': torch.tensor(count_blanks, dtype=torch.long),
            'next_to_vowel': torch.tensor(blank_vowel_next, dtype=torch.float)
        }
    
class EnhancedHangmanModel(nn.Module):
    def __init__(self, vocab_size=28, max_len=45, emb_dim=128, hidden_dim=1024, ablate={}):
        super().__init__()
        self.ablate = ablate
        self.char_emb = nn.Embedding(vocab_size, emb_dim)
        self.ctx_emb = nn.Embedding(4, 32)

        self.pattern_cnn = nn.Sequential(
            nn.Conv1d(emb_dim, 64, 3, padding=1), nn.ReLU(), nn.Dropout(0.2),
            nn.Conv1d(64, 64, 3, padding=1), nn.ReLU(), nn.Dropout(0.2)
        )

        self.encoder = nn.LSTM(emb_dim + 32, hidden_dim, bidirectional=True, batch_first=True)

        self.pos_prior_mlp = nn.Sequential(
            nn.Linear(1 + 1 + 64, 32), nn.ReLU(), nn.Dropout(0.2), nn.Linear(32, 26)
        )

        def decoder():
            return nn.Sequential(
                nn.Linear(hidden_dim*2 + 26, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3),

                nn.Linear(hidden_dim, hidden_dim // 2),
                nn.ReLU(),
                nn.Dropout(0.3),

                nn.Linear(hidden_dim // 2, 26)
            )

        self.left_decoder = decoder()
        self.right_decoder = decoder()
        self.both_decoder = decoder()

    def forward(self, word_state, position_context, word_length, blank_count, next_to_vowel):
        B, L = word_state.size()
        emb = self.char_emb(word_state)
        cnn_feat = self.pattern_cnn(emb.transpose(1, 2)).transpose(1, 2)
        ctx = self.ctx_emb(position_context)
        encoded, _ = self.encoder(torch.cat([emb, ctx], -1))

        pos_scores = []
        for i in range(L):
            is_blank = (word_state[:, i] == 0).float().unsqueeze(1)
            bc = blank_count.unsqueeze(1).float() / L
            pos_input = torch.cat([is_blank, bc, cnn_feat[:, i, :]], -1)
            pos_scores.append(self.pos_prior_mlp(pos_input).unsqueeze(1))
        priors = torch.cat(pos_scores, 1)  # [B, L, 26]

        out = torch.zeros(B, L, 26, device=word_state.device)
        for i in range(L):
            h = encoded[:, i, :]
            ptype = position_context[:, i]
            inp = torch.cat([h, priors[:, i, :]], -1)
            out[ptype==1, i, :] = self.left_decoder(inp[ptype==1])
            out[ptype==2, i, :] = self.right_decoder(inp[ptype==2])
            out[ptype==3, i, :] = self.both_decoder(inp[ptype==3])

        return out

class HangmanSolver:
    def __init__(self, word_list, model_path="best_model.pth"):
        self.model = EnhancedHangmanModel()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.load_state_dict(torch.load(model_path, map_location=self.device, weights_only=True))
        if torch.cuda.device_count() > 1:
            self.model = nn.DataParallel(self.model)

        self.model.eval()

        self.dictionary = word_list
        self.char_to_idx = {c: i+1 for i, c in enumerate(string.ascii_lowercase)}
        self.char_to_idx['_'] = 0
        self.idx_to_char = {v: k for k, v in self.char_to_idx.items() if v != 0}

    def _fallback_prediction(self, pattern, guessed):
        counter = Counter()
        for word in self.dictionary:
            if len(word) != len(pattern):
                continue
            match = True
            for wc, pc in zip(word, pattern):
                if pc != '_' and pc != wc:
                    match = False
                    break
                if pc == '_' and wc in guessed:
                    match = False
                    break
            if match:
                for i, c in enumerate(word):
                    if pattern[i] == '_' and c not in guessed:
                        counter[c] += 1

        if not counter:
            for c in string.ascii_lowercase:
                if c not in guessed:
                    return c
            return random.choice([c for c in string.ascii_lowercase if c not in guessed])  # final fallback

        for letter, _ in counter.most_common():
            if letter not in guessed:
                return letter

        return random.choice([c for c in string.ascii_lowercase if c not in guessed])

    def predict_letter(self, word_state, guessed_letters=None):
        if guessed_letters is None:
            guessed_letters = set()
        if ' ' in word_state:
            word_state = word_state.replace(' ', '')
        if all(c == '_' for c in word_state):
            # Return most frequent unguessed letter as the first guess
            common_order = "etaoinshrdlucmfwypvbgkjqxz"
            for letter in common_order:
                if letter not in guessed_letters:
                    return letter
        word_state = word_state.lower()
        max_length = 45
        state_indices = []
        position_context = []

        for i, char in enumerate(word_state):
            if char == '_':
                state_indices.append(0)
                context = 0
                if i > 0 and word_state[i-1] != '_':
                    context += 1
                if i < len(word_state)-1 and word_state[i+1] != '_':
                    context += 2
                position_context.append(context)
            else:
                state_indices.append(self.char_to_idx.get(char, 27))
                position_context.append(0)

        while len(state_indices) < max_length:
            state_indices.append(27)
            position_context.append(0)

        word_tensor = torch.tensor([state_indices], dtype=torch.long).to(self.device)
        context_tensor = torch.tensor([position_context], dtype=torch.long).to(self.device)
        length_tensor = torch.tensor([len(word_state)], dtype=torch.long).to(self.device)
        blank_count_tensor = torch.tensor([word_state.count('_')], dtype=torch.long).to(self.device)
        blank_vowel_next = [0]*max_length

        for i in range(len(word_state)):
            if word_state[i] == '_':
                l = word_state[i-1] if i > 0 else 'x'
                r = word_state[i+1] if i < len(word_state)-1 else 'x'
                if l in 'aeiou' or r in 'aeiou':
                    blank_vowel_next[i] = 1

        blank_vowel_tensor = torch.tensor([blank_vowel_next], dtype=torch.float).to(self.device)

        with torch.no_grad():
            predictions = self.model(word_tensor, context_tensor, length_tensor, blank_count_tensor, blank_vowel_tensor)

        best_predictions = []
        for i in range(len(word_state)):
            if word_state[i] == '_' and position_context[i] > 0:
                probs = torch.softmax(predictions[0, i, :], dim=0)
                for j, prob in enumerate(probs):
                    letter = chr(ord('a') + j)
                    if letter not in guessed_letters:
                        best_predictions.append((letter, prob.item(), i))

        if not best_predictions:
            for i in range(len(word_state)):
                if word_state[i] == '_':
                    probs = torch.softmax(predictions[0, i, :], dim=0)
                    for j, prob in enumerate(probs):
                        letter = chr(ord('a') + j)
                        if letter not in guessed_letters:
                            best_predictions.append((letter, prob.item(), i))

        if best_predictions:
            best_predictions.sort(key=lambda x: x[1], reverse=True)
            return best_predictions[0][0]
        
        # Fallback to frequency-based guess if model fails
        common_order = "etaoinshrdlucmfwypvbgkjqxz"
        for letter in common_order:
            if letter not in guessed_letters:
                return letter

        return 'e'  # very rare fallback if all else fails

from tqdm import tqdm

def train_model(words, epochs=10, early_stopping_patience=5):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = EnhancedHangmanModel()
    if torch.cuda.device_count() > 1:
        model = nn.DataParallel(model)
    model.to(device)

    opt = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    scheduler = ReduceLROnPlateau(opt, patience=2)
    loss_fn = nn.CrossEntropyLoss()

    best = float('inf')
    patience_counter = 0

    for ep in range(epochs):
        # Curriculum: reveal_ratio increases with epoch (starts hard, becomes easier)
        reveal_ratio = max(0.1, 1.0 - (ep+1) * 0.067)  # Start from 1.0, decrease to 0.1
        ds = HangmanDataset(words, reveal_ratio=reveal_ratio)
        train_len = int(0.9 * len(ds))
        tr, val = random_split(ds, [train_len, len(ds)-train_len])
        dl = DataLoader(tr, shuffle=True, pin_memory=True, batch_size=256, num_workers=4)
        vl = DataLoader(val, pin_memory=True, batch_size=256, num_workers=4)

        print(f"\n--- Epoch {ep+1} | Reveal Ratio: {reveal_ratio:.2f} ---", flush=True)
        model.train()
        total_loss = 0
        batch_count = 0

        for i, batch in enumerate(tqdm(dl, desc="Training", ncols=100)):
            opt.zero_grad()
            out = model(batch['word_state'].to(device), batch['position_context'].to(device),
                        batch['word_length'].to(device), batch['blank_count'].to(device),
                        batch['next_to_vowel'].to(device))
            loss, count = 0, 0
            for b in range(out.size(0)):
                target_pos = batch['target_positions'][b].to(device)
                target_char = batch['target_chars'][b].to(device)
                for p, c in zip(target_pos, target_char):
                    if p >= 0 and c > 0:
                        loss += loss_fn(out[b, p], c-1)
                        count += 1
            if count > 0:
                loss = loss / count
                loss.backward()
                opt.step()
                total_loss += loss.item()
                batch_count += 1
            if i % 20 == 0:
                if isinstance(loss, torch.Tensor):
                    print(f"  Batch {i}/{len(dl)} | Loss: {loss.item():.4f}", flush=True)
                else:
                    print(f"  Batch {i}/{len(dl)} | Loss: N/A (no valid targets)", flush=True)


        train_loss = total_loss / batch_count if batch_count > 0 else 0
        model.eval()
        val_loss = 0
        val_batches = 0
        with torch.no_grad():
            for batch in tqdm(vl, desc="Validation", ncols=100):
                out = model(batch['word_state'].to(device), batch['position_context'].to(device),
                            batch['word_length'].to(device), batch['blank_count'].to(device),
                            batch['next_to_vowel'].to(device))
                loss, count = 0, 0
                for b in range(out.size(0)):
                    target_pos = batch['target_positions'][b].to(device)
                    target_char = batch['target_chars'][b].to(device)
                    for p, c in zip(target_pos, target_char):
                        if p >= 0 and c > 0:
                            loss += loss_fn(out[b, p], c-1)
                            count += 1
                if count > 0:
                    val_loss += loss.item() / count
                    val_batches += 1

        val_loss = val_loss / val_batches if val_batches > 0 else 0
        scheduler.step(val_loss)

        if count > 0 and i % 20 == 0:
            print(f"  Batch {i}/{len(dl)} | Loss: {loss.item():.4f}", flush=True)

        if val_loss < best:
            best = val_loss
            patience_counter = 0
            torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), "best_model.pth")
            print("✅ Model improved and saved.", flush=True)
        else:
            patience_counter += 1
            print(f"⚠️ No improvement. Patience: {patience_counter}/{early_stopping_patience}", flush=True)
            if patience_counter >= early_stopping_patience:
                print(f"🛑 Early stopping at epoch {ep+1}", flush=True)
                break

    return model

In [3]:
dictionary = open("words_250000_train.txt").read().splitlines()
random.shuffle(dictionary)
word_list = dictionary[:10000]
#model = train_model(word_list, epochs=15)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model1 = EnhancedHangmanModel()
#torch.save(model.module.state_dict() if isinstance(model, nn.DataParallel) else model.state_dict(), "best_model.pth")
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model1 = nn.DataParallel(model1)

model1 = model1.to(device)
model_path = "best_model1.pth"
checkpoint = torch.load(model_path, map_location=device, weights_only=True)

if isinstance(model1, nn.DataParallel):
    model1.module.load_state_dict(checkpoint)
else:
    model1.load_state_dict(checkpoint)
solver1 = HangmanSolver(word_list, model_path="best_model1.pth")
solver1.model = model1
solver1.model.eval()
#------------------------------------
model2 = EnhancedHangmanModel()
if torch.cuda.device_count() > 1:
    print(f"Using {torch.cuda.device_count()} GPUs")
    model2 = nn.DataParallel(model2)

model2 = model2.to(device)
model_path0 = "best_model2.pth"
checkpoint0 = torch.load(model_path0, map_location=device, weights_only=True)

if isinstance(model2, nn.DataParallel):
    model2.module.load_state_dict(checkpoint)
else:
    model2.load_state_dict(checkpoint0)
solver2 = HangmanSolver(word_list, model_path="best_model2.pth")
solver2.model = model2
solver2.model.eval()

EnhancedHangmanModel(
  (char_emb): Embedding(28, 128)
  (ctx_emb): Embedding(4, 32)
  (pattern_cnn): Sequential(
    (0): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Conv1d(64, 64, kernel_size=(3,), stride=(1,), padding=(1,))
    (4): ReLU()
    (5): Dropout(p=0.2, inplace=False)
  )
  (encoder): LSTM(160, 1024, batch_first=True, bidirectional=True)
  (pos_prior_mlp): Sequential(
    (0): Linear(in_features=66, out_features=32, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=32, out_features=26, bias=True)
  )
  (left_decoder): Sequential(
    (0): Linear(in_features=2074, out_features=1024, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.3, inplace=False)
    (3): Linear(in_features=1024, out_features=512, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.3, inplace=False)
    (6): Linear(in_features=512, out_features=26, bias=True)
  )
  (right_decoder): Seque

In [ ]:
import tkinter as tk
from tkinter import messagebox, simpledialog


class HangmanGame:
    def __init__(self, master):
        self.master = master
        self.master.title("Hangman AI Game")
        self.solver1 = solver1
        self.solver2 = solver2
        self.word_state = []
        self.guessed_letters = set()
        self.attempts = 0
        self.max_attempts = 6  # Adjusted to traditional hangman
        self.word_length = 0
        self.setup_screen()

    def setup_screen(self):
        self.intro_frame = tk.Frame(self.master)
        self.intro_frame.pack(pady=20)
        tk.Label(self.intro_frame, text="Enter the length of your secret word:", font=("Helvetica", 14)).pack()
        self.length_entry = tk.Entry(self.intro_frame, font=("Helvetica", 14))
        self.length_entry.pack()
        tk.Button(self.intro_frame, text="Start Game", font=("Helvetica", 14), command=self.start_game).pack(pady=10)

    def start_game(self):
        try:
            self.word_length = int(self.length_entry.get())
            if self.word_length <= 0:
                raise ValueError
        except ValueError:
            messagebox.showerror("Invalid Input", "Please enter a valid positive integer.")
            return
        self.word_state = ['_'] * self.word_length
        self.intro_frame.destroy()
        self.create_game_ui()

    def create_game_ui(self):
        self.canvas = tk.Canvas(self.master, width=200, height=250, bg='white')
        self.canvas.pack(pady=10)
        self.draw_hangman()

        self.word_label = tk.Label(self.master, text=" ".join(self.word_state), font=("Helvetica", 24))
        self.word_label.pack(pady=10)

        self.info_label = tk.Label(self.master, text="Guessed Letters: None\nAttempts Left: 6", font=("Helvetica", 12))
        self.info_label.pack()

        tk.Button(self.master, text="Let AI Guess", font=("Helvetica", 14), command=self.ai_guess).pack(pady=10)

    def update_ui(self):
        self.word_label.config(text=" ".join(self.word_state))
        self.info_label.config(
            text=f"Guessed Letters: {', '.join(sorted(self.guessed_letters)) or 'None'}\nAttempts Left: {self.max_attempts - self.attempts}"
        )
        self.draw_hangman()

    def get_reveal_ratio(self):
        total = len(self.word_state)
        revealed = sum(1 for c in self.word_state if c != '_')
        return revealed / total if total > 0 else 0

    def ai_guess(self):
        if '_' not in self.word_state or self.attempts >= self.max_attempts:
            return
        solver = self.solver1 if self.get_reveal_ratio() > 0.5 else self.solver2
        guess = solver.predict_letter(''.join(self.word_state), self.guessed_letters)
        self.guessed_letters.add(guess)
        answer = messagebox.askyesno("AI Guess", f"Does the letter '{guess}' appear in your word?")
        if answer:
            positions = simpledialog.askstring("Correct Guess", f"Enter 1-based positions of '{guess}' separated by commas (e.g. 1,3):")
            if positions:
                try:
                    indices = [int(i.strip()) - 1 for i in positions.split(',')]
                    for idx in indices:
                        if 0 <= idx < self.word_length:
                            self.word_state[idx] = guess
                except:
                    messagebox.showerror("Error", "Invalid positions entered.")
        else:
            self.attempts += 1
        self.update_ui()
        if '_' not in self.word_state:
            messagebox.showinfo("Game Over", "🎉 The AI guessed your word!")
        elif self.attempts >= self.max_attempts:
            messagebox.showinfo("Game Over", "💀 The AI failed to guess your word.")

    def draw_hangman(self):
        self.canvas.delete("all")
        # Gallows
        self.canvas.create_line(20, 230, 180, 230)
        self.canvas.create_line(50, 230, 50, 20)
        self.canvas.create_line(50, 20, 120, 20)
        self.canvas.create_line(120, 20, 120, 40)

        if self.attempts > 0:
            self.canvas.create_oval(100, 40, 140, 80)  # Head
        if self.attempts > 1:
            self.canvas.create_line(120, 80, 120, 140)  # Body
        if self.attempts > 2:
            self.canvas.create_line(120, 100, 90, 120)  # Left arm
        if self.attempts > 3:
            self.canvas.create_line(120, 100, 150, 120)  # Right arm
        if self.attempts > 4:
            self.canvas.create_line(120, 140, 90, 180)  # Left leg
        if self.attempts > 5:
            self.canvas.create_line(120, 140, 150, 180)  # Right leg

def run_hangman_gui():
    root = tk.Tk()
    game = HangmanGame(root)
    root.mainloop()

run_hangman_gui()
